In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('./../augmented_reviews.csv')
df.head()

,Reviews,labels
0,गोवा में उर्फ ​​रिकी बहल ब्लड बस्टर्ड की खोज का संदर्भ लंबा और ऊब गया है।,negative
1,पुरानी फिल्म के प्रति नॉस्टेलजिक होना ठीक है।,neutral
2,"सोनिया के प्यार के एक दृश्य में, सामाजिक कार्यकर्ता मनीष राजकुमार राव मृणाल ठाकुर को बचाने के लिए जीवन देता है और उसे बचाने के लिए वेश्यालय तक भी पहुंचता है, लेकिन सोनिया जिस्मफारोशी के उस विचित्रता से बाहर निकलने के लिए सहमत नहीं है। इसका कारण यह है कि उसकी छोटी बहन रिया रियासिसोदिया, जो उसे बचाने के लिए इस काली दुनिया में आए हैं। डाइरेक्टर तबरेज़ नूरानी ने दो बहनों के बहाने मानव यातायात की बदसूरत दुनिया की क्रूर और क्रूर सच्चाई को उजागर किया है, जो आप किसान के कर्ज को देखकर विचलित हो जाते हैं।कर्ज का बर्डन। फार्मर शिव आदिल हुसैन ने अपनी बेटी प्रीति को मानव तस्करी के तहत दादा ठाकुर अनुपम खेर के साथ अपने कर्ज को निपटाने के लिए साइन किया।यहाँ बहन, वह भी एक ही जाल में फंस जाती है। वेश्यालय के मालिक, वेश्यालय के मालिक, मणोज बाजपेयी, लड़कियों को पेशेवर रूप से प्राप्त करने के लिए सभी डरावनी रणनीति को अपनाते हैं।यह कुछ बहाने पर दलदल है। कहानी में मोड़ तब आता है जब सोनिया अपनी बहन को पाता है। यह सोनिया खुद को और बहन को वेश्यावृत्ति की इस भयानक दुनिया से बाहर कर देता है, इसके लिए आपको फिल्म देखना होगा। बॉलीवुड में, यहां तक ​​कि शरीर के व्यापार में भी, यहां तक ​​कि शरीर के व्यापार में भी,मंडी बाज़ार मौसम लक्ष्मी बेगम जान जैसी फिल्में बन गए हैं, लेकिन निर्देशक तबरेज नूरानी की दिशा से पता चलता है कि उन्होंने द ब्लैक वर्ल्ड ऑफ बॉडी ट्रेड का गहरा अध्ययन किया है। फिल्म की कहानी को सच्ची घटनाओं पर आधारित कहा जाता है।मानवता के अंधेरे पक्षों को बहुत अच्छी तरह से दिखाया है कि कैसे लोग अपने स्वार्थ और अर्थ के लिए असंवेदनशील हो जाते हैं। सोनिया का एक दृश्य है जो तितली को एक जार में रखता है जो पूरी फिल्म की प्रकृति का वर्णन करता है।फिल्म में विदेशों में मानव तस्करी का विस्तार भी होना चाहिए। फिल्म के कास्टिंग और चरित्र दिलचस्प हैं। फिल्म अभिनय में बहुत समृद्ध है। पहली फिल्म होने के नाते, डेब्यूटेंट मृणाल ठाकुर ने इतनी जबरदस्त अभिनय किया है कि आप असहनीय कचोट होने लगते हैं।ऋचा चड्हा का चरित्र मानव के रूप में निर्दयी है। मन्नोज़ बाजपेयी ने फिजल के रूप में दृढ़ता से प्रदर्शन किया है। फ़्रीडा पिंटो अपनी भूमिका में प्रभाव डालता है। साई तम्हंकर राजकुमार राव अनुपम खेर आदिल हुसैन ने अपने चरित्रों के अनुसार बहुत अच्छा प्रदर्शन किया है।डेमी मूर को सुखद लगता है। फिल्म का पृष्ठभूमि स्कोर काफी दिलचस्प है जो कहानी से मेल खाती है। क्यों यह फिल्म इस फिल्म को यथार्थवादी डार्क फिल्मों के शौकीन देख सकती है।",negative
3,उन्होंने इस फिल्म के भावनात्मक विस्फोट के दृश्यों को अधिक नहीं दिया।,positive
4,विक्रम छठे दशक के बंगाल के संदर्भ में एक दिलचस्प और उदास प्रेम कहानी चुनता है।,positive


In [3]:
df.isna().sum()

Reviews    0
labels     0
dtype: int64

In [10]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")


Using device: cuda


In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import BertTokenizer
from polyglot.text import Text

In [11]:
# Load your dataset (assuming CSV format with columns 'text' and 'label')
data = df.copy()
texts = data['Reviews'].tolist()
labels = data['labels'].tolist()

# Split into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load the mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def polyglot_tokenize(text):
    polyglot_text = Text(text, hint_language_code='hi')  # Tokenize with Polyglot
    return polyglot_text.words  # Return tokens as a list

def custom_tokenize(texts, tokenizer, max_length=128):
    all_input_ids = []
    for text in texts:
        # Step 1: Tokenize using Polyglot
        polyglot_tokens = polyglot_tokenize(text)

        # Step 2: Map Polyglot tokens to mBERT vocabulary
        input_ids = []
        for token in polyglot_tokens:
            token_id = tokenizer.convert_tokens_to_ids(token)
            if token_id is None:  # Handle tokens not in mBERT's vocabulary
                token_id = tokenizer.unk_token_id  # Use [UNK] token ID
            input_ids.append(token_id)

        # Step 3: Truncate sequences if necessary
        if len(input_ids) > max_length:
            input_ids = input_ids[:max_length]

        all_input_ids.append(input_ids)

    # Step 4: Pad sequences to the maximum length
    encoded = tokenizer.pad(
        {"input_ids": all_input_ids},
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoded


train_encodings = custom_tokenize(train_texts, tokenizer)
test_encodings = custom_tokenize(test_texts, tokenizer)



label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2} # Create a mapping
train_labels = [label_mapping[label] for label in train_labels] # Apply mapping
test_labels = [label_mapping[label] for label in test_labels]


# Create Dataset objects for Trainer
class HindiTextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = HindiTextDataset(train_encodings, train_labels)
test_dataset = HindiTextDataset(test_encodings, test_labels)

# Load the mBERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(set(labels)))
model.to(device)

# Define multiple metrics for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Set up Trainer with early stopping
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=22,
    per_device_eval_batch_size=22,
    num_train_epochs=60,  # Max epochs
    weight_decay=0.01,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,  # Limit saved checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Evaluate on test dataset
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],  
)

# Train the model
trainer.train()


/home/lenovo/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2852: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lenovo/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-763863ad85b1>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.026144,0.475806,0.414747,0.502317,0.475806
2,1.045700,0.981093,0.493952,0.496627,0.504465,0.493952
3,1.045700,0.953522,0.549059,0.542783,0.548658,0.549059
4,0.903300,0.917337,0.577285,0.573727,0.577723,0.577285
5,0.903300,0.877550,0.622312,0.623180,0.627116,0.622312
6,0.728000,0.828420,0.647177,0.648154,0.653457,0.647177
7,0.728000,0.841001,0.669355,0.670244,0.674387,0.669355
8,0.526600,1.006898,0.680108,0.668979,0.702852,0.680108
9,0.526600,0.979812,0.705645,0.706499,0.711198,0.705645
10,0.350300,1.041849,0.698253,0.700324,0.709436,0.698253


<ipython-input-11-763863ad85b1>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-763863ad85b1>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-763863ad85b1>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-763863ad85b1>:62: UserWarning: To copy construct fro

TrainOutput(global_step=14905, training_loss=0.16941646229137075, metrics={'train_runtime': 7316.9973, 'train_samples_per_second': 48.807, 'train_steps_per_second': 2.222, 'total_flos': 2.153320210685952e+16, 'train_loss': 0.16941646229137075, 'epoch': 55.0})

In [6]:
# Evaluate the model on the test set
test_results = trainer.evaluate(test_dataset)
print("Test Evaluation Results:", test_results)

<ipython-input-5-a9009bde3882>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Test Evaluation Results: {'eval_loss': 1.1041765213012695, 'eval_accuracy': 0.8649193548387096, 'eval_f1': 0.8646433553096033, 'eval_precision': 0.8649455810283394, 'eval_recall': 0.8649193548387096, 'eval_runtime': 11.009, 'eval_samples_per_second': 135.162, 'eval_steps_per_second': 6.177, 'epoch': 30.0}
